In [ ]:
from pathlib import Path

import pandas as pd

from pairs_trading_etf.data.ingestion import download_etf_data, save_raw_data, validate_price_data
from pairs_trading_etf.utils.config import load_yaml_config

CONFIG_PATH = Path("configs/data.yaml")
RAW_OUTPUT = Path("data/raw/etf_prices.csv")

config = load_yaml_config(CONFIG_PATH)
tickers = config["universe"]["etfs"]
date_cfg = config["data"]

prices = download_etf_data(
    tickers=tickers,
    start=date_cfg["start_date"],
    end=date_cfg["end_date"],
)

save_raw_data(prices, RAW_OUTPUT)
summary = validate_price_data(prices)

print(f"Downloaded price frame shape: {prices.shape}")
display(prices.head())
summary

## Next steps
- [ ] Compute correlation matrix & visualize via heatmap.
- [ ] Run rolling correlation to identify stable relationships.
- [ ] Implement Engle–Granger two-step test for top candidate pairs.
- [ ] Validate cointegration hits with Johansen test (if multivariate).